### Load Data

In [1]:
import pandas as pd
import re
import sqlite3

In [2]:
messages = pd.read_csv('./data/raw/messages.csv')
categories = pd.read_csv('./data/raw/categories.csv')

In [3]:
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [4]:
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [5]:
messages.genre.value_counts()

news      13068
direct    10782
social     2398
Name: genre, dtype: int64

### Prepare categories

In [6]:
def get_last_digit(string):
    return string[-1]

def categories_to_dummy(df, column='categories', delim=';'):
    new_df = df.copy()
    new_df_expanded = new_df.categories.str.split(delim, expand=True)
    new_df = new_df[['id']].merge(new_df_expanded, left_index=True, right_index=True)
    column_names =  new_df.iloc[0,1:]
    
    pattern = re.compile('-\d')
    for i, name in enumerate(column_names):
        column_names[i] = re.sub(pattern, '', name)
    new_df.columns = ['id'] + column_names.tolist()
    

    new_df.iloc[:, 1:] = new_df.iloc[:, 1:].applymap(get_last_digit)
    
    return new_df

In [7]:
categories = categories_to_dummy(categories)

### Merge with messages

In [8]:
full_data = messages.merge(categories, on='id')

In [9]:
print('Duplicate rows: {}'.format(full_data.shape[0] - full_data.drop_duplicates().shape[0]))

Duplicate rows: 170


In [10]:
full_data.drop_duplicates(inplace=True)

### Post Processing

In [11]:
# Column "related" contains value 2 for some reason
full_data.related.replace('2', '1', inplace=True)

### Save as DB File

In [12]:
conn = sqlite3.connect('messages_info.db')
c = conn.cursor()

full_data.to_sql(name='data', con=conn, if_exists='replace')